In [1]:
import matplotlib
matplotlib.use('Agg')

In [2]:
import numpy as np
import pandas as pd
import os  
import seaborn as sns
import matplotlib.pyplot as plt
import csv
import io
import dateutil
import collections
import time
from datetime import datetime
import wrds
import networkx as nx

In [3]:
nx.__version__

'2.2'

In [ ]:
db = wrds.Connection()

In [75]:
list_libraries = db.list_libraries()
list_libraries[:10]

['aha_sample',
 'ahasamp',
 'audit',
 'audit_audit_comp',
 'audit_corp_legal',
 'block',
 'boardex',
 'boardex_na',
 'boardex_trial',
 'bvd']

In [4]:
nodelist = pd.read_excel('Supply_Chain_No_Dup.xlsx') #make a pandas df from Nodes.csv
supplierinfo = pd.read_excel('Supplier_Names.xlsx') #Supplier Names with gvkey
customerinfo = pd.read_excel('Customer_Names.xlsx') #Customer Names with gvkey

In [5]:
print(list(nodelist))
#edge_list[:10]

['SupplierGvkey', 'SupplierName', 'CustomerGvkey', 'CustomerCurrentName', 'Date', 'Sales']


In [10]:
#Create Graph and Edges from dataframe
G=nx.from_pandas_edgelist(nodelist,'CustomerGvkey','SupplierGvkey',edge_attr=True)
#from_pandas_d_edgelist(df,source,target,edge_attr=True (true pulls all other columns))

In [12]:
for i, nlrow in supplierinfo.iterrows():
    G.node[nlrow['SupplierGvkey']].update(nlrow[1:].to_dict())  #Add info to Supplier Gvkey Nodes.


In [13]:
for i, nlrow in customerinfo.iterrows():
    G.node[nlrow['CustomerGvkey']].update(nlrow[1:].to_dict()) #Add info to Customer Gvkey Nodes

Had to change the three cells above because of networkx 2.2

See urls: https://networkx.github.io/documentation/stable/reference/generated/networkx.convert_matrix.from_pandas_edgelist.html#networkx.convert_matrix.from_pandas_edgelist

https://stackoverflow.com/questions/47045155/typeerror-nodeview-object-does-not-support-item-assignment-networkx

In [14]:
nx.number_of_nodes(G)

11735

In [256]:
type(G.degree())  # the number of degrees for each node

networkx.classes.reportviews.DegreeView

In [24]:
#for line in nx.generate_edgelist(G, data=False):
    #print(line)

The above cell is how to generate an edge list. I am unsure how to do just the first 10.

In [257]:
nx.connected_components(G) #Connected nodes

<generator object connected_components at 0x7fa205ff85a0>

In [26]:
degrees = nx.degree_centrality(G) #Degrees of nodes

In [27]:
closeness = nx.closeness_centrality(G) #Closeness of nodes

In [28]:
betweeness = nx.betweenness_centrality(G) #Betweenness for nodes

In [258]:
#betweeness

In [30]:
clustering = nx.clustering(G) #Clustering of nodes

In [259]:
#clustering

In [344]:
degdf = pd.DataFrame()
degdf["Gvkey"] = degrees.keys()
degdf["degree"] = [x[1] for x in list(G.degree())]
print(degdf.shape)
subgraphdf = degdf.sort_values(by="degree",ascending=False)[:1153].dropna()

(11735, 2)


In [177]:
companylist = supplierinfo
companylist = companylist.rename(index=str, columns={"SupplierGvkey": "Gvkey","CompanyName":"Company_Name"})
clist = customerinfo
clist = clist.rename(index=str, columns={'CustomerGvkey': "Gvkey","CurrentCompanyName":"Company_Name"})
companylist = companylist.append(clist)

In [180]:
companylist = companylist.drop_duplicates()

In [347]:
subgraphdf = subgraphdf.reset_index().drop(["index"],axis=1)
companylist = companylist.reset_index().drop(["index"],axis=1)

In [309]:
print(subgraphdf[:10])

      nodeid  degree  Clustering  Betweeness  Closeness Company_Name
7655   25747     570    0.000000    0.000076   0.226684         None
2547    6100     453    0.036364    0.000185   0.242023         None
2100    5091     416    0.666667    0.000003   0.231981         None
510     1585     382    0.000000    0.000847   0.228509         None
1992    4856     333    0.000000    0.000163   0.151751         None
2646    6331     325    0.133333    0.000104   0.221431         None
2085    5061     275    0.000000    0.000000   0.226756         None
4157    9570     228    0.000000    0.000049   0.215787         None
3246    7609     221    0.000000    0.000000   0.191688         None
2333    5626     211    0.000000    0.000000   0.168071         None


In [345]:
subgraphdf["Clustering"] = 0
subgraphdf["Betweeness"] = 0
subgraphdf["Closeness"] = 0

In [346]:
for i in subgraphdf.index:
    subgraphdf.loc[i, 'Clustering'] = clustering[subgraphdf["Gvkey"][i]]
    subgraphdf.loc[i, 'Betweeness'] = betweeness[subgraphdf["Gvkey"][i]]
    subgraphdf.loc[i, 'Closeness'] = closeness[subgraphdf["Gvkey"][i]]

In [335]:
print(subgraphdf[0:10])
print(companylist[0:10])

   Gvkey  degree  Clustering  Betweeness  Closeness Company_Name
0  25747     570    0.000000    0.000076   0.226684         None
1   6100     453    0.036364    0.000185   0.242023         None
2   5091     416    0.666667    0.000003   0.231981         None
3   1585     382    0.000000    0.000847   0.228509         None
4   4856     333    0.000000    0.000163   0.151751         None
5   6331     325    0.133333    0.000104   0.221431         None
6   5061     275    0.000000    0.000000   0.226756         None
7   9570     228    0.000000    0.000049   0.215787         None
8   7609     221    0.000000    0.000000   0.191688         None
9   5626     211    0.000000    0.000000   0.168071         None
   Gvkey                Company_Name
0   1005       A.B.A. INDUSTRIES INC
1   1009          ABS INDUSTRIES INC
2   1012          ACS INDUSTRIES INC
3   1013  ADC TELECOMMUNICATIONS INC
4   1021            AFP IMAGING CORP
5   1023           AGS COMPUTERS INC
6   1028          AIM TELE

In [348]:
subgraphdf_names = pd.merge(subgraphdf,companylist, left_on="Gvkey", right_on="Gvkey")

In [350]:
subgraphdf_names

,Gvkey,degree,Clustering,Betweeness,Closeness,Company_Name
0,25747,570,0.000000,7.643913e-05,0.226684,LITTELFUSE INC
1,6100,453,0.036364,1.852795e-04,0.242023,INTERDIGITAL INC
2,5091,416,0.666667,2.992381e-06,0.231981,VERIZON INC/FL
3,1585,382,0.000000,8.466678e-04,0.228509,AMERICAN VANGUARD CORP
4,4856,333,0.000000,1.627746e-04,0.151751,FORUM GROUP INC
5,6331,325,0.133333,1.043273e-04,0.221431,FURMANITE CORP
6,5061,275,0.000000,0.000000e+00,0.226756,GENERAL HOUSEWARES
7,9570,228,0.000000,4.878629e-05,0.215787,ABC DISPENSING TECHNOLOGIES
8,7609,221,0.000000,0.000000e+00,0.191688,MULTIMEDIA INC
9,5626,211,0.000000,0.000000e+00,0.168071,HIGH PLAINS CORP


In [110]:
deg_freq = degdf.groupby('degree').count()
#plt.plot(deg_freq.index, np.log(deg_freq['nodeid']))
deg_freq2 = deg_freq.sort_index(ascending=False)
deg_freq2['cumfreq'] = np.cumsum(deg_freq2.nodeid)
#deg_freq2.tail(125)